##### [Gemini Code]

In [ ]:
import subprocess
import json
import sys
import time

# --- CONFIGURATION ---

# 1. This command MUST exactly match the 'command' and 'args' from your mcpServers JSON.
# Ensure Node.js and npx are accessible in your environment.
MCP_COMMAND: list[str] = ["npx", "-y", "@kimtaeyoon83/mcp-server-youtube-transcript"]

# 2. Define a known, short, publicly available video URL for testing.
TEST_URL: str = "https://www.youtube.com/watch?v=R9K13r_p-y8"

# 3. Define the function call payload (the agent's request)
# Note: The exact 'method' name must be correct for the server implementation.
TEST_PAYLOAD: dict = {
    "id": 1,
    # This is a common function name for the transcript server
    "method": "youtube_transcript_extractor",
    "params": {
        "url": TEST_URL
    }
}

# --- TEST EXECUTION ---

print(f"🚀 Starting MCP Server test with command: {' '.join(MCP_COMMAND)}")
# 

try:
    # Launch the server as a subprocess, capturing its input/output
    process = subprocess.Popen(
        MCP_COMMAND,
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True  # Handles encoding for JSON strings
    )

    # Convert the Python dictionary to a JSON string and add a newline
    json_payload = json.dumps(TEST_PAYLOAD) + "\n"

    # Send the payload to the server's stdin and wait for output
    stdout_data, stderr_data = process.communicate(input=json_payload, timeout=20)
    
    # 1. Check for errors printed directly to stderr (Node errors, installation issues)
    if stderr_data:
        # Ignore common npx warning about package location
        if "npm install" not in stderr_data:
            print(f"Server Error (stderr): {stderr_data}", file=sys.stderr)

    # 2. Check for successful JSON response on stdout
    if not stdout_data:
         print("MCP Server Test FAILED: Server returned no output (might be hanging or crashed).")
         
    response = json.loads(stdout_data.strip())

    # 3. Check the content of the response
    if 'result' in response:
        print("MCP Server Test SUCCESS!")
        transcript = response['result']
        
        print(transcript)
            
    elif 'error' in response:
        print(f"MCP Server Test FAILED (Function Error): {response['error']}")
        
    else:
        print("MCP Server Test FAILED: Unexpected JSON response format.")
        
except Exception as e:
    print(f"An unexpected Python error occurred: {e}")

finally:
    # Ensure the process is cleaned up
    if 'process' in locals() and process.poll() is None:
        process.terminate()
        time.sleep(1)
        process.kill()

In [ ]:
import asyncio
import os
import sys
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

# --- CONFIGURATION ---
# Choose your server command here.
# Option A: Python-based 'mcp-youtube-extract' (ensure it's in your PATH or use absolute path)
SERVER_COMMAND = "mcp_youtube_extract"
SERVER_ARGS = []
SERVER_ENV = os.environ.copy()

# Option B: Node-based 'zubeid-youtube-mcp-server' (Requires Node.js installed)
# SERVER_COMMAND = "npx"
# SERVER_ARGS = ["-y", "zubeid-youtube-mcp-server"]
# SERVER_ENV["YOUTUBE_API_KEY"] = "YOUR_GOOGLE_API_KEY_HERE"
# ---------------------

async def run_youtube_client():
    # 1. Configure the server connection parameters
    server_params = StdioServerParameters(
        command=SERVER_COMMAND,
        args=SERVER_ARGS,
        env=SERVER_ENV
    )

    print(f"Starting server: {SERVER_COMMAND}...")

    # 2. Connect to the server via Standard Input/Output (stdio)
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # 3. Initialize the session
            await session.initialize()
            
            # 4. List available tools (Optional, helps you see what you can do)
            print("\n--- Available Tools ---")
            tools = await session.list_tools()
            for tool in tools.tools:
                print(f"- {tool.name}: {tool.description}")

            # 5. Call a tool to retrieve data
            # Example video: "Never Gonna Give You Up"
            video_url = "https://www.youtube.com/watch?v=dQw4w9WgXcQ"
            
            print(f"\n--- Fetching Data for: {video_url} ---")
            
            # Note: Tool names depend on the server you use. 
            # 'mcp-youtube-extract' uses 'get_yt_video_info'
            # 'zubeid-youtube-mcp-server' uses 'get_video_details'
            tool_name = "get_yt_video_info" 
            
            try:
                result = await session.call_tool(
                    tool_name,
                    arguments={"video_id": "dQw4w9WgXcQ"} # Or "url": video_url depending on tool
                )
                
                # 6. Print the result
                # The result usually comes back as a list of content blocks (text/images)
                for content in result.content:
                    if content.type == "text":
                        print(content.text)
                        
            except Exception as e:
                print(f"Error calling tool: {e}")

if __name__ == "__main__":
    # Windows compatibility for asyncio
    if sys.platform == "win32":
        asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
        
    asyncio.run(run_youtube_client())

In [2]:
from mcp.client import Client

client = Client.spawn("mcp-youtube")  # auto‑spawns the process
path = client.download_video(
    url="https://youtu.be/dQw4w9WgXcQ",
    quality="best",
    format="mp4",
    resolution="1080p",
)
print("Saved to", path)

ImportError: cannot import name 'Client' from 'mcp.client' (c:\Users\PK\anaconda3\envs\python-new\Lib\site-packages\mcp\client\__init__.py)